## Roteiro de como o script funciona.
1. Selecionar IDs válidos de paciente, unidadeSaude, e profissionalSaude a partir do banco de dados.
2. Gerar 300 atendimentos com data e hora distribuídas nos primeiros 3 meses de 2018.
3. Inserir esses atendimentos no banco de dados, garantindo que os IDs utilizados são válidos.

In [ ]:
import psycopg2
import random
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

# Carregar variáveis do arquivo .env
load_dotenv()

# Pegar as credenciais do arquivo .env

# Obter variáveis de ambiente
db_host = os.getenv('DB_HOST')
db_database = os.getenv('DB_DATABASE')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_port = os.getenv('DB_PORT')

# Conectar ao banco de dados
conn = psycopg2.connect(
    host=db_host,
    database=db_database,
    user=db_user,
    password=db_password,
    port=db_port
)
cursor = conn.cursor()

# Função para buscar IDs (UUIDs) válidos no banco de dados
def buscar_uuids_tabela(tabela, coluna):
    cursor.execute(f"SELECT {coluna} FROM {tabela}")
    return [row[0] for row in cursor.fetchall()]

# Função para gerar datas distribuídas nos primeiros 3 meses de 2018
def gerar_data_aleatoria():
    start_date = datetime(2018, 1, 1)
    end_date = datetime(2018, 3, 31)
    delta = end_date - start_date
    random_days = random.randint(0, delta.days)
    random_time = timedelta(hours=random.randint(8, 18), minutes=random.randint(0, 59))  # Entre 08:00 e 18:59
    return start_date + timedelta(days=random_days) + random_time

# Função para gerar 300 atendimentos
def gerar_atendimentos(num_atendimentos):
    ids_unidade = buscar_uuids_tabela('unidadeSaude', 'id')
    ids_paciente = buscar_uuids_tabela('paciente', 'id')
    ids_profissional = buscar_uuids_tabela('profissionalSaude', 'id')

    atendimentos = []
    for _ in range(num_atendimentos):
        id_unidade = random.choice(ids_unidade)
        id_paciente = random.choice(ids_paciente)
        id_profissional = random.choice(ids_profissional)

        # Simular busca de nomes e CBO correspondentes ao profissional e unidade
        cursor.execute(f"SELECT nome, CBO FROM profissionalSaude WHERE id = %s", (id_profissional,))
        profissional_data = cursor.fetchone()
        nome_profissional = profissional_data[0]
        CBO_profissional = profissional_data[1]

        cursor.execute(f"SELECT nome FROM unidadeSaude WHERE id = %s", (id_unidade,))
        nome_unidade = cursor.fetchone()[0]

        cursor.execute(f"SELECT nome FROM paciente WHERE id = %s", (id_paciente,))
        nome_paciente = cursor.fetchone()[0]

        # Gerar data e hora aleatória dentro dos primeiros 3 meses de 2018
        data_hora_atendimento = gerar_data_aleatoria()

        # CID aleatório para simplificação (pode adaptar conforme necessidade)
        CID = f"A{random.randint(0, 99):02d}"

        atendimentos.append((
            id_unidade, nome_unidade, id_paciente, nome_paciente, 
            id_profissional, nome_profissional, CBO_profissional, 
            data_hora_atendimento, CID
        ))
    return atendimentos

# Função para inserir os atendimentos no banco
def inserir_atendimentos(atendimentos):
    insert_query = """
    INSERT INTO atendimento (id_unidade, nome_unidade, id_paciente, nome_paciente, 
    id_profissionalSaude, nome_profissionalSaude, CBO_profissional, data_hora_atendimento, CID) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.executemany(insert_query, atendimentos)
    conn.commit()

# Gerar 300 atendimentos e inserir no banco
atendimentos_gerados = gerar_atendimentos(300)
inserir_atendimentos(atendimentos_gerados)

# Fechar a conexão com o banco de dados
cursor.close()
conn.close()

print("Atendimentos inseridos com sucesso!")


## Drescrevendo o codigo passo a passo

Gerar o DataFrame:
O script agora retorna um DataFrame com os 300 atendimentos gerados, permitindo que você visualize os dados antes de qualquer inserção no banco de dados.

Use df_atendimentos.head() para exibir os primeiros 5 registros e posteriormente o Data Wrangler para 
validar os dados.

In [ ]:
import psycopg2
import random
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import pandas as pd

# Carregar variáveis do arquivo .env
load_dotenv()

# Pegar as credenciais do arquivo .env
db_host = os.getenv('DB_HOST')
db_database = os.getenv('DB_DATABASE')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_port = os.getenv('DB_PORT')

# Conectar-se ao banco de dados PostgreSQL usando as variáveis do .env
conn = psycopg2.connect(
    host=db_host,
    database=db_database,
    user=db_user,
    password=db_password,
    port=db_port
)
cursor = conn.cursor()

# Função para buscar IDs (UUIDs) válidos no banco de dados
def buscar_uuids_tabela(tabela, coluna):
    cursor.execute(f"SELECT {coluna} FROM {tabela}")
    return [row[0] for row in cursor.fetchall()]

# Função para gerar datas distribuídas nos primeiros 3 meses de 2018
def gerar_data_aleatoria():
    start_date = datetime(2018, 1, 1)
    end_date = datetime(2018, 3, 31)
    delta = end_date - start_date
    random_days = random.randint(0, delta.days)
    random_time = timedelta(hours=random.randint(8, 18), minutes=random.randint(0, 59))  # Entre 08:00 e 18:59
    return start_date + timedelta(days=random_days) + random_time

# Função para gerar 300 atendimentos e retornar um DataFrame
def gerar_atendimentos(num_atendimentos):
    ids_unidade = buscar_uuids_tabela('unidadeSaude', 'id')
    ids_paciente = buscar_uuids_tabela('paciente', 'id')
    ids_profissional = buscar_uuids_tabela('profissionalSaude', 'id')

    atendimentos = []
    for _ in range(num_atendimentos):
        id_unidade = random.choice(ids_unidade)
        id_paciente = random.choice(ids_paciente)
        id_profissional = random.choice(ids_profissional)

        # Simular busca de nomes e CBO correspondentes ao profissional e unidade
        cursor.execute(f"SELECT nome, CBO FROM profissionalSaude WHERE id = %s", (id_profissional,))
        profissional_data = cursor.fetchone()
        nome_profissional = profissional_data[0]
        CBO_profissional = profissional_data[1]

        cursor.execute(f"SELECT nome FROM unidadeSaude WHERE id = %s", (id_unidade,))
        nome_unidade = cursor.fetchone()[0]

        cursor.execute(f"SELECT nome FROM paciente WHERE id = %s", (id_paciente,))
        nome_paciente = cursor.fetchone()[0]

        # Gerar data e hora aleatória dentro dos primeiros 3 meses de 2018
        data_hora_atendimento = gerar_data_aleatoria()

        # CID aleatório para simplificação (pode adaptar conforme necessidade)
        CID = f"A{random.randint(0, 99):02d}"

        atendimentos.append([
            id_unidade, nome_unidade, id_paciente, nome_paciente, 
            id_profissional, nome_profissional, CBO_profissional, 
            data_hora_atendimento, CID
        ])
    
    # Criar um DataFrame para validar os dados
    colunas = ['id_unidade', 'nome_unidade', 'id_paciente', 'nome_paciente', 
               'id_profissionalSaude', 'nome_profissionalSaude', 'CBO_profissional', 
               'data_hora_atendimento', 'CID']
    df_atendimentos = pd.DataFrame(atendimentos, columns=colunas)
    
    return df_atendimentos

# Gerar 300 atendimentos e retornar como DataFrame
df_atendimentos = gerar_atendimentos(300)

# Exibir os primeiros 5 atendimentos para validação
df_atendimentos.head()



### Validação:
Você pode inspecionar o DataFrame para garantir que todas as informações dos atendimentos (unidades, pacientes, profissionais, datas, e CID) estão corretas.

Pode também aplicar outros métodos do pandas como df_atendimentos.describe() ou df_atendimentos.info() para explorar os dados.

In [ ]:
import psycopg2.extras as extras

# Função para inserir os atendimentos no banco a partir do DataFrame
def inserir_atendimentos_db(df_atendimentos):
    conn = psycopg2.connect(
        host=db_host,
        database=db_database,
        user=db_user,
        password=db_password,
        port=db_port
    )
    cursor = conn.cursor()

    # Preparar os dados para inserção
    atendimentos = df_atendimentos.values.tolist()
    
    insert_query = """
    INSERT INTO atendimento (id_unidade, nome_unidade, id_paciente, nome_paciente, 
    id_profissionalSaude, nome_profissionalSaude, CBO_profissional, data_hora_atendimento, CID) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    # Usar execute_batch para inserir os dados de forma eficiente
    extras.execute_batch(cursor, insert_query, atendimentos)
    
    # Confirmar a inserção e fechar a conexão
    conn.commit()
    cursor.close()
    conn.close()

# Após validar os dados no DataFrame, insira no banco
inserir_atendimentos_db(df_atendimentos)


In [5]:
cursor.close()
conn.close()

### CONTROLANDO AS INSERCOES

Inserindo dados dentro de um limiar, maior do que 25 atendimentos diários por 6 meses

In [1]:
import psycopg2
import random
from datetime import timedelta
from dotenv import load_dotenv
import os
import pandas as pd

# Carregar variáveis do arquivo .env
load_dotenv()

# Pegar as credenciais do arquivo .env
db_host = os.getenv('DB_HOST')
db_database = os.getenv('DB_DATABASE')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_port = os.getenv('DB_PORT')

# Conectar-se ao banco de dados PostgreSQL usando as variáveis do .env
conn = psycopg2.connect(
    host=db_host,
    database=db_database,
    user=db_user,
    password=db_password,
    port=db_port
)
cursor = conn.cursor()

# Função para buscar IDs (UUIDs) válidos no banco de dados
def buscar_uuids_tabela(tabela, coluna):
    cursor.execute(f"SELECT {coluna} FROM {tabela}")
    return [row[0] for row in cursor.fetchall()]

def buscar_uuids_tabelaPaciente(tabela, coluna,sexo):
    cursor.execute(f"SELECT {coluna} FROM {tabela} WHERE sexo = %s", (sexo,))
    return [row[0] for row in cursor.fetchall()]

# Função para gerar uma hora aleatória dentro de um dia específico
def gerar_hora_aleatoria_no_dia(dia):
    random_time = timedelta(hours=random.randint(8, 18), minutes=random.randint(0, 59))  # Horário entre 08:00 e 18:59
    return dia + random_time

# Função para gerar atendimentos contínuos durante 6 meses, respeitando os dias de cada mês
def gerar_atendimentos_por_periodo(start_date, end_date):
    ids_unidade = buscar_uuids_tabela('unidadeSaude', 'id')
    ids_paciente = buscar_uuids_tabelaPaciente('paciente', 'id', 'F')
    ids_profissional = buscar_uuids_tabela('profissionalSaude', 'id')

    atendimentos = []
    
    # Gerar todas as datas do intervalo usando pandas para lidar com o número correto de dias de cada mês
    dias_periodo = pd.date_range(start=start_date, end=end_date, freq='D')

    # Gerar atendimentos para cada dia no intervalo
    for current_date in dias_periodo:
        num_atendimentos_por_dia = random.randint(36, 42)  # Número aleatório de atendimentos por dia
        for _ in range(num_atendimentos_por_dia):
            id_unidade = random.choice(ids_unidade)
            id_paciente = random.choice(ids_paciente)
            id_profissional = random.choice(ids_profissional)

            # Simular busca de nomes e CBO correspondentes ao profissional e unidade
            cursor.execute(f"SELECT nome, CBO FROM profissionalSaude WHERE id = %s", (id_profissional,))
            profissional_data = cursor.fetchone()
            nome_profissional = profissional_data[0]
            CBO_profissional = profissional_data[1]

            cursor.execute(f"SELECT nome FROM unidadeSaude WHERE id = %s", (id_unidade,))
            nome_unidade = cursor.fetchone()[0]

            cursor.execute(f"SELECT nome FROM paciente WHERE id = %s", (id_paciente,))
            nome_paciente = cursor.fetchone()[0]

            # Gerar uma hora aleatória dentro do dia atual
            data_hora_atendimento = gerar_hora_aleatoria_no_dia(current_date)

            # CID aleatório para simplificação (pode adaptar conforme necessidade)
            CID = f"A{random.randint(0, 99):02d}"

            atendimentos.append([
                id_unidade, nome_unidade, id_paciente, nome_paciente, 
                id_profissional, nome_profissional, CBO_profissional, 
                data_hora_atendimento, CID
            ])
    
    # Criar um DataFrame para validar os dados
    colunas = ['id_unidade', 'nome_unidade', 'id_paciente', 'nome_paciente', 
               'id_profissionalSaude', 'nome_profissionalSaude', 'CBO_profissional', 
               'data_hora_atendimento', 'CID']
    df_atendimentos = pd.DataFrame(atendimentos, columns=colunas)
    
    return df_atendimentos

# Definir a data de início e fim do período
start_date = '2018-07-01'
end_date = '2018-07-31'

# Gerar atendimentos e retornar como DataFrame
df_atendimentos = gerar_atendimentos_por_periodo(start_date, end_date)

# Exibir os primeiros 5 atendimentos para validação
df_atendimentos.head()

,id_unidade,nome_unidade,id_paciente,nome_paciente,id_profissionalSaude,nome_profissionalSaude,CBO_profissional,data_hora_atendimento,CID
0,3b3b0223-7607-46b7-a5c8-6f90d0092b15,Unidade de Saúde Central,31dd43f2-f264-4ccd-93db-cc340fc82a34,Milena da Mata,f0a1f2bd-a21d-40e5-a15f-a74bb7c9927e,Tec. Rafael Santos,3222-01,2018-07-01 08:06:00,A51
1,d817a1ff-2523-4691-b895-d64c9ad16ac3,Unidade de Saúde Zona Sul,a0962a2e-ae5f-4c44-ac0f-18fa6f11db71,Oliver Machado,ddaef559-55de-418b-b279-2943746fe39d,Tec. Amanda Rocha,3222-01,2018-07-01 10:26:00,A53
2,d817a1ff-2523-4691-b895-d64c9ad16ac3,Unidade de Saúde Zona Sul,1089fad2-862d-422d-b82d-0da3c0d24e3f,Sra. Bruna Peixoto,b52b9f7d-5b7e-497f-989c-133683f19954,Enf. Ana Pereira,2235-01,2018-07-01 13:37:00,A53
3,3b3b0223-7607-46b7-a5c8-6f90d0092b15,Unidade de Saúde Central,727d62e1-13b9-41fe-9412-24d5e9ffe7d5,Olívia Aragão,e4d1ad76-ea68-4e6b-a740-bb64c802432f,Tec. José Costa,3222-01,2018-07-01 12:29:00,A60
4,d817a1ff-2523-4691-b895-d64c9ad16ac3,Unidade de Saúde Zona Sul,e7bce159-bd02-4807-9798-f3059c503d1c,Mariane Pimenta,6bb14e38-4e5b-4b18-b9bf-d070e836cc64,Dr. Carlos Nunes,2251-01,2018-07-01 13:18:00,A54


### VALIDACAO 02-

Apos validado agora só inserir novo modelo na base

In [2]:
import psycopg2.extras as extras

# Função para inserir os atendimentos no banco a partir do DataFrame
def inserir_atendimentos_db(df_atendimentos):
    conn = psycopg2.connect(
        host=db_host,
        database=db_database,
        user=db_user,
        password=db_password,
        port=db_port
    )
    cursor = conn.cursor()

    # Preparar os dados para inserção
    atendimentos = df_atendimentos.values.tolist()
    
    insert_query = """
    INSERT INTO atendimento (id_unidade, nome_unidade, id_paciente, nome_paciente, 
    id_profissionalSaude, nome_profissionalSaude, CBO_profissional, data_hora_atendimento, CID) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    # Usar execute_batch para inserir os dados de forma eficiente
    extras.execute_batch(cursor, insert_query, atendimentos)
    
    # Confirmar a inserção e fechar a conexão
    conn.commit()
    cursor.close()
    conn.close()

# Após validar os dados no DataFrame, insira no banco
inserir_atendimentos_db(df_atendimentos)
